# Dummy Example

This notebook shows the use of the dummy feedback function provider which
behaves like the huggingface provider except it does not actually perform any
network calls and just produces constant results. It can be used to prototype
feedback function wiring for your apps before invoking potentially slow (to
run/to load) feedback functions.

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.resolve()))

In [2]:
import threading

from examples.frameworks.custom.custom_app import CustomApp

from trulens_eval import Feedback
from trulens_eval import FeedbackMode
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.feedback.provider.hugs import Huggingface
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.resolve()
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Deleted 10 rows.
Force stopping dashboard ...
killing psutil.Process(pid=97214, name='python3.8', status='running', started='11:36:46')
Starting dashboard ...


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.86.103:8501 .


In [3]:
# hugs = Huggingface()
hugs = Dummy()

f_positive_sentiment = Feedback(hugs.positive_sentiment).on_output()

✅ In positive_sentiment, input text will be set to *.__record__.main_output or `Select.RecordOutput` .


In [4]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # feedback_mode=FeedbackMode.WITH_APP
    feedbacks=[f_positive_sentiment]
)

In [5]:
# Test of a sequence of records:
"""
with ta:
    for i, q in enumerate(["hello there"] * 100):
        print(f"\rrequest {i} ", end="")
        print(f"thread count={threading.active_count()}, promises={TP().promises.qsize()}", end="")

        res = ca.respond_to_query(input=q)
"""

'\nwith ta:\n    for i, q in enumerate(["hello there"] * 100):\n        print(f"\rrequest {i} ", end="")\n        print(f"thread count={threading.active_count()}, promises={TP().promises.qsize()}", end="")\n\n        res = ca.respond_to_query(input=q)\n'

In [6]:
from ipywidgets import interact
from ipywidgets import widgets
from IPython.display import JSON
from trulens_eval.utils.serial import JSONPath
from typing import Any, Callable
from traitlets import Unicode, Bool, validate, TraitError, HasTraits
import traitlets
from ipywidgets import DOMWidget, register

cstyle = dict(border="1px solid black", padding="5px")

class Selector(HasTraits):
    select = Unicode()
    jpath = traitlets.Any()

    def __init__(self, select: str, make_on_delete: Callable):
        self.select = select
        self.jpath = JSONPath.of_string(select)

        self.w_edit = widgets.Text(value=select)
        self.w_delete = widgets.Button(description="x", layout=dict(width="30px"))

        self.on_delete = make_on_delete(self)
        self.w_delete.on_click(self.on_delete)

        traitlets.link((self.w_edit, "value"), (self, "select"))

        def on_update_select(ev):
            jpath = JSONPath.of_string(ev.new)
            self.jpath = jpath
        self.observe(on_update_select, ["select"])

        self.w = widgets.HBox([self.w_delete, self.w_edit])

    
class SelectorValue(HasTraits):
    selector = traitlets.Any()
    obj = traitlets.Any()

    def __init__(self, selector: Selector):
        self.selector = selector
        self.obj = None

        self.w_listing = widgets.HTML()
        self.w = widgets.VBox([self.selector.w, self.w_listing])

        def update_selector(ev):
            v = ev.new
            # self.
            pass

        self.selector.w_edit.observe(update_selector, "value")

    def set_obj(self, obj):
        self.obj = obj

        if obj is None:
            v = "no listing yet"
        else:
            try:
                v = str(next(iter(self.selector.jpath(obj.dict()))))[0:16]
            except Exception as e:
                v = str(e)

        self.w_listing.value = f"<div>{v}</div>"


class RecordWiget():
    def __init__(self, record_selections, record=None, human_or_input=None):
        self.record = record
        self.record_selections = record_selections
        self.record_values = dict()

        self.human_or_input = widgets.HBox([human_or_input])
        self.w_human = widgets.HBox([widgets.HTML("<b>human:</b>"), self.human_or_input])
        self.d_comp = widgets.HTML(layout=cstyle)
        self.d_extras = widgets.VBox(layout=cstyle)
        self.d = None

        self.human = ""
        self.comp = ""

        self.draw_main()

    def update_selections(self):
        for s in self.record_selections:
            if s not in self.record_values:
                sv = SelectorValue(selector=s)
                self.record_values[s] = sv
                self.d_extras.children += (sv.w, )

            self.record_values[s].set_obj(obj=self.record)

    def remove_selector(self, selector: Selector):
        item = self.record_values[selector]
        del self.record_values[selector]
        new_children = list(self.d_extras.children)
        new_children.remove(item.w)
        self.d_extras.children = tuple(new_children)


    def set_human(self, human: str):
        self.human = human
        self.human_or_input.children = (widgets.HTML(f"<div>{human}</div>"), )

    def set_comp(self, comp: str):
        self.comp = comp
        self.d_comp.value = f"<div><b>computer:</b> {comp}</div>"

    def draw_main(self):
        self.d = widgets.VBox([self.w_human, self.d_comp, self.d_extras], layout=cstyle)
        

In [7]:
main_input = widgets.Text(continuous_update=False, layout=cstyle)
app_selector = widgets.Text(continuous_update=False, layout=cstyle)
record_selector = widgets.Text(continuous_update=False, layout=cstyle)

display_top = widgets.Output(layout=cstyle)
display_side = widgets.VBox([], layout=cstyle)
display_middle = widgets.Output(layout=cstyle)

display_records = []

app_selections = {}
record_selections = []

current_record = RecordWiget(record_selections=record_selections, human_or_input=main_input)
current_record_record = None

records = [current_record]

def make_on_delete_record_selector(selector):
    def on_delete(ev):
        print("delete record selector", str(selector.jpath))

        record_selections.remove(selector)

        for r in records:
            r.remove_selector(selector)

    return on_delete

def make_on_delete_app_selector(selector):
    def on_delete(ev):
        print("delete app selector", str(selector.jpath))

        sw = app_selections[selector]
        del app_selections[selector]

        new_children = list(display_side.children)
        new_children.remove(sw.w)

        display_side.children = tuple(new_children)

    return on_delete


def update_app_selections():
    for s, sw in app_selections.items():
        sw.set_obj(ta)


def add_app_selection(ev):
    s = ev.new

    if len(s) == 0:
        return

    sel = Selector(select=s, make_on_delete=make_on_delete_app_selector)

    sw = SelectorValue(selector=sel)
    app_selections[sel] = sw
    sw.set_obj(ta)

    display_side.children += (sw.w, )

def add_record_selection(ev):
    s = ev.new

    if len(s) == 0:
        return
    
    sel = Selector(select=s, make_on_delete=make_on_delete_record_selector)
    record_selections.append(sel)

    for r in records:
        r.update_selections()

def runit(ev):
    global current_record
    global current_record_record

    human = ev.new

    current_record.set_human(human)

    with ta as recording:
        comp = ca.respond_to_query(input=human)

    current_record.set_comp(comp)
    current_record_record = recording.get()
    current_record.record = current_record_record
    current_record.update_selections()

    update_app_selections()

    current_record = RecordWiget(record_selections=record_selections, human_or_input=main_input)
    records.append(current_record)

    update_app_selections()

    with display_top:
        display(current_record.d)

main_input.observe(runit, ["value"])
app_selector.observe(add_app_selection, ["value"])
record_selector.observe(add_record_selection, ["value"])

display_bottom = widgets.VBox([
    widgets.HBox([widgets.Label("main_input:", layout=cstyle), main_input], layout=cstyle),
    widgets.HBox([widgets.Label("app select:", layout=cstyle), app_selector], layout=cstyle),
    widgets.HBox([widgets.Label("rec select:", layout=cstyle), record_selector], layout=cstyle)],
    layout=cstyle
)

display(
    widgets.HBox([
        widgets.VBox([
            display_top,
            display_bottom
        ], layout=cstyle),
        display_middle,
        display_side], layout=cstyle)
    )

with display_top:
    display(current_record.d)


Waiting for {'estimated_time': 1.2345} (1.2345) second(s).


delete record selector *.cost
